# Gender inference attack using recommendations - UserKNN

1. Data loading

In [1]:
import pandas as pd

# recommendations
recs_df = pd.read_csv('../recs/UserKNN.tsv', header=None, sep='\t')
recs_df.rename(columns={0: 'uid', 1: 'movie_id', 2: 'score', 3: 'timestamp'}, inplace=True)
recs_df.set_index('uid')
recs_df.head(5)

,uid,movie_id,score
0,1,595,3.688725
1,1,2716,3.386360
2,1,910,3.191516
3,1,1196,3.127495
4,1,593,3.096445


In [2]:
# users
user_df = pd.read_csv('../data/users.tsv', header=None, sep='\t')
user_df.head(5)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


filter out user attributes (maintain only id and age)

In [3]:
user_age_df = user_df.iloc[:, : 3]
user_age_df.rename(columns={0: 'uid', 1: 'gender', 2: 'age'}, inplace=True)
user_age_df = user_age_df.drop('gender', 1)
user_age_df.set_index('uid')
user_age_df.head(5)

/var/folders/0r/m1t8hv6s7l9bsyqckwntb9qm0000gn/T/ipykernel_3723/2521958798.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  user_age_df = user_age_df.drop('gender', 1)


,uid,age
0,1,1
1,2,56
2,3,25
3,4,45
4,5,25


2. join datasets

In [4]:
joined_df = pd.merge(user_age_df, recs_df, on='uid')
joined_df.head(5)

,uid,age,movie_id,score
0,1,1,595,3.688725
1,1,1,2716,3.386360
2,1,1,910,3.191516
3,1,1,1196,3.127495
4,1,1,593,3.096445


3. format dataset for classification task

In [5]:
joined_df.rename(columns={'age': 'class'}, inplace=True)
joined_df['class'].unique()

array([ 1, 56, 25, 45, 50, 35, 18])

4. training/test set splitting

In [6]:
from sklearn.model_selection import train_test_split

dff = joined_df
x = dff.iloc[:, dff.columns != 'class']
y = dff.iloc[:, dff.columns == 'class']
X_train, X_test, y_train, y_test = train_test_split (x,y, test_size = 0.3, random_state = 0)

5. classify (random forest)

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

/var/folders/0r/m1t8hv6s7l9bsyqckwntb9qm0000gn/T/ipykernel_3723/1687212718.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


6. evaluation metrics (random forest)

In [8]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: " + str(accuracy))

Accuracy: 0.5362030905077263


7. classification (logistic regression)

In [9]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
pred = logisticRegr.predict(X_test)

/Users/mauromarini/PycharmProjects/privacy_and_fairness_in_recsys/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/mauromarini/PycharmProjects/privacy_and_fairness_in_recsys/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


8. evaluation metrics (Logistic regression)

In [10]:
accuracy_lr = accuracy_score(y_test, pred)
print("Accuracy: " + str(accuracy_lr))

Accuracy: 0.3483443708609272
